In [ ]:
import os, sys
os.chdir('/home/inarighas/Projects/voice-transcript')
import pandas as pd
import numpy as np
import librosa
from bs4 import BeautifulSoup
from app.main import process_transcription

import json
import wave
import contextlib
import fastwer

In [ ]:
root_folder = '../voice-features/eval/data/ParoleParisContemporain/'
folder = 'record-28'

In [ ]:
# res = []
# for folder in os.listdir(root_folder)[:1]:
#     if folder.startswith("record"):
#         print(folder)
#         list_files = os.listdir(root_folder + folder)
#         for f in list_files:
#             if f.endswith('.trs'):
#                 with open(os.path.join(root_folder, folder, f), 'tr') as tmp:
#                     data = tmp.read()
#                 bs_data = BeautifulSoup(data, 'xml')
#                 speakers = bs_data.find("Speakers")
#                 list_speakers = [s.get('id') for s in speakers if len(s)==0]
#                 timestamps = {s: [] for s in list_speakers}
#                 for s in list_speakers:
#                     for tag in bs_data.find_all('Turn', {'speaker':s}):
#                         tmp = tag.attrs
#                         tmp["text"] = tag.text.replace("\n", "")
#                         res.append(tmp)

In [ ]:
# with open(os.path.join(root_folder, folder, "transcript.json"), 'w', encoding='utf8') as f:
#     json.dump(res, f, indent=2, ensure_ascii=False)

In [ ]:
with open(os.path.join(root_folder, folder, "transcript.json"), 'r') as f:
    d = json.load(f)

In [ ]:
target_sr = 8000
name = os.path.join(root_folder, folder, "single_spk1.wav")
arr, sr =  librosa.load(name)
    # get list of audio(31.2*sr)]
    # list_audio = [np.array(audio[i:i + sr*60]) for i in range(0, audio.shape[0], sr*60)]
    # for idx, audio in enumerate(list_audio):
    # Run transcript for each audio subsample:
    # save retext, dur = process_transcription(arr, sr)sults


In [ ]:
offset = 0
for item in d:
    length = float(item['endTime']) - float(item['startTime'])
    speech_array = librosa.resample(arr[offset: offset + int(length*sr)],
                                    orig_sr=sr,
                                    target_sr=target_sr,
                                    res_type="kaiser_fast"
                                    )
    text, dur = process_transcription(speech_array, target_sr)
    item['model'] = text
    # print(f"Offset: {offset} ---- Length: {int(length*sr)}")
    offset = offset + int(length*sr)

In [ ]:
import json
with open(os.path.join(root_folder, folder, "res_transcript.json"), 'w', encoding='utf8') as f:
    json.dump(d, f, indent=2, ensure_ascii=False)

In [ ]:
with open(os.path.join(root_folder, folder, "res_transcript.json"), 'r') as f:
    d = json.load(f)

In [ ]:
ref_list = []
mod_list = []
for idx, i in enumerate(d):
    try:
        mod_list.append(i['model'])
        ref_list.append(i['text'])
    except KeyError:
        print(f"Line {idx}: unfound transcript")
    

In [ ]:
len(mod_list)

In [ ]:
wer_lst = [fastwer.score_sent(m, r) for m, r in zip(mod_list, ref_list)]
cer_lst = [fastwer.score_sent(m, r, char_level=True) for m, r in zip(mod_list, ref_list)]

print('\n---------------------------------------------------')
print(f"Sentence level WER: \t avg {np.array(wer_lst).mean():.2f} (std {np.array(wer_lst).std():.2f})")
print(f"Corpus level WER: \t {fastwer.score(mod_list, ref_list, char_level=False):.2f}")
print('---------------------------------------------------')
print(f"Sentence level CER: \t avg {np.array(cer_lst).mean():.2f} (std {np.array(cer_lst).std():.2f})")
print(f"Corpus level CER: \t {fastwer.score(mod_list, ref_list, char_level=True):.2f}")

In [ ]:
import seaborn as sns
sns.histplot(wer_lst, color='k', alpha=.5)
sns.histplot(cer_lst, color='red', alpha=.5)